## Exposing your tools from the MCP server

If you want to expose your tools or data so that other agents would be able to access it, then you could leverage MCP servers. Databricks already provides [managed MCP servers](https://docs.databricks.com/aws/en/generative-ai/mcp/managed-mcp#available-managed-servers), so that you do not need to start from scratch. 


These managed servers include the UC functions, vector search indexes, and Genie spaces. In our example, we will show how you can provide the agent, the tools from the Databricks Managed MCP servers such as the UC system ai functions, but in similar fashion you can configure it for any other either Databricks managed MCP server or custom. 

In [0]:
%pip install -U -qqqq mlflow>=3.1.1 langchain langgraph databricks-langchain pydantic databricks-agents databricks-mcp  unitycatalog-langchain[databricks] uv databricks-feature-engineering==0.12.1
dbutils.library.restartPython()

In [0]:
%run ../_resources/01-setup

## Databricks Managed MCP

To use the UC system ai function from the Databricks Managed MCP server, we'd just need to add the relevant url in the [agent_config.yaml](https://adb-984752964297111.11.azuredatabricks.net/editor/files/2254899697811646?o=984752964297111) file.

- **Unity Catalog system ai functions**: https://{workspace-hostname}/api/2.0/mcp/functions/system/ai

In [0]:
import nest_asyncio
nest_asyncio.apply()

import mlflow
import yaml, sys, os
import mlflow.models

# ==========================
# Add agent_eval folder to path
# ==========================
agent_eval_path = os.path.abspath(os.path.join(os.getcwd(), "../02-agent-eval"))
sys.path.append(agent_eval_path)

conf_path = os.path.join(agent_eval_path, 'agent_config.yaml')

# ==========================
# Update YAML safely to include MCP server
# ==========================
with open(conf_path, "r") as f:
    config = yaml.safe_load(f)

# Only update serializable fields
config["config_version_name"] = "model_with_mcp"
config["mcp_server_urls"] = [
    "https://adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/functions/system/ai"
]

# Write YAML safely
with open(conf_path, "w") as f:
    yaml.safe_dump(config, f)

In [0]:
# Use same experiment to keep all traces together
#--------Here it exceeds the recursion depth
#mlflow.set_experiment(agent_eval_path + "/02.1_agent_evaluation")

## Test the agent

To make sure we have provided exposed the tools from the managed MCP server to the agent, we will do this by checking all the available tools that the agent has. In the list, should also be appearing the tools from MCP with the UC system ai functions i.e **system__ai__python_exec**

In [0]:
from agent import LangGraphResponsesAgent, ModelConfig
from mlflow.types import responses as resp

model_config = mlflow.models.ModelConfig(development_config=conf_path)

AGENT = LangGraphResponsesAgent(
    uc_tool_names=model_config.get("uc_tool_names"),
    llm_endpoint_name=model_config.get("llm_endpoint_name"),
    system_prompt=model_config.get("system_prompt"),
    retriever_config=model_config.get("retriever_config"),
    mcp_server_urls=model_config.get("mcp_server_urls"),
    max_history_messages=model_config.get("max_history_messages"),
)

print("Available tools:", AGENT.list_tools())

In [0]:
import mlflow.types.responses as resp

user_code = "result = 2 + 2\nprint(f'The result is {result}')"
request_example = f"Execute Python code: {user_code}"

request = resp.ResponsesAgentRequest(
    input=[{"role": "user", "content": request_example}]
)

response = AGENT.predict(request)

print("=== Agent Output ===")
for item in response.output:
    print(item)